### This script generates the automatic metric scores for all the model submissions (in the canonical GLUCOSE test set format)

In [10]:
!pip install pandas
!pip3 install sacrebleu

In [11]:
import sacrebleu

In [12]:
import glob, os
import re
import pandas
from collections import defaultdict

In [13]:
# This block can be commented out if Google Drive is not used
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Set the paths





In [14]:
# Note: change the following paths to point to your model submission folders and the GLUCOSE 'test_set_answer_key.csv' file.
submissions_path = "/content/drive/My Drive/GLUCOSE Colab Notebooks/submissions/"
key_file_path = "/content/drive/My Drive/GLUCOSE Colab Notebooks/test_set/test_set_answer_key"

## Calculate the metrics
We are only caclulating BLEU scores here

In [24]:
model_evaluation_results = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

In [22]:
df_key = pandas.read_csv(key_file_path, encoding="utf-8")
for path in glob.glob(os.path.join('', submissions_path+"*.csv")):
  model_name = path.replace(submissions_path,"").replace(".csv", "")
  df_generation = pandas.read_csv(path, encoding="utf-8")
  for dim in range(10):
    dim = dim + 1
    for mode in ["specific", "general"]:
      predictions = df_generation[str(dim)+"_"+mode+"NL"].values
      references = [[], [], []] #3 references per each test case
      for one_row in df_key[str(dim)+"_"+mode+"NL"].values:
        if one_row != "escaped":
          splitted = one_row.split("****")
          for i in range(3):            
            references[i].append(splitted[i])
        else:
          for i in range(3):
            references[i].append("escaped")
      bleu = sacrebleu.corpus_bleu(predictions, references).score
      model_evaluation_results["dim_"+str(dim)][mode][model_name]["bleu"] = bleu

## Print the results

In [25]:
model_evaluation_results

defaultdict(<function __main__.<lambda>>, {})